In [1]:
# Prevent pycache from being generated while developing
import sys
sys.dont_write_bytecode=True

In [2]:
# Import required packages
import ee
from pprint import pprint

In [3]:
# Import local files
import config
import geelayers
import cities
import helpers
import super_rasters

KeyboardInterrupt: 

In [4]:
# Reload local files while developing
import importlib
# importlib.reload(cities)
importlib.reload(geelayers)
# importlib.reload(config)
# importlib.reload(helpers)
# importlib.reload(super_rasters)

<module 'geelayers' from '/home/weiqi_tori/GitHub/wri/urban_extent/gee/geelayers.py'>

In [5]:
# Get list of image collections in a folder
def get_collections(folder):
    collections = ee.data.getList({'id': folder})
    return collections


collections = get_collections(config.ROOT)


## Select Image collection
### Count the number of images in the collection
size = len(collections)
print(size)

### Extract image names as a list
collection_ids = [collection['id'] for collection in collections]
pprint(collection_ids)

16
['projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/GHSL_BUthresh10pct_JRCs_50compare_L2_20_wP_2020',
 'projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/GHSL_BUthresh10pct_JRCs_50compare_L2_20_wP_2020_circle',
 'projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/builtup_density_JRCs_1000',
 'projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/builtup_density_JRCs_1000_circle',
 'projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/builtup_density_JRCs_50',
 'projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/builtup_density_JRCs_50compare_L1',
 'projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/builtup_density_JRCs_50compare_L2',
 'projec

In [6]:
# Create map
collection = 'projects/earthengine-legacy/assets/projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/builtup_density_JRCs_1000'

#importlib.reload(geelayers)
geelayers.interactive_map(collection)

Dropdown(description='Pass Scale Factor Check:', options=('All', 'True', 'False'), style=DescriptionStyle(desc…

Dropdown(description='Select Scale Factor:', options=('All', 20, 50, 80, 120, 200, 400, 800, 2000), style=Desc…

Dropdown(description='Select Image:', options=('Abidjan-1675-2020', 'Abilene-264-2020', 'Abington-920-2020', '…

Dropdown(description='Select Band:', options=('builtup_class', 'density', 'builtup'), style=DescriptionStyle(d…

Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [8]:
size = geelayers.GHSL_2020.size().getInfo()
print(f"Image Collection Size: {size}")
print(f'DEST: {config.IC_ID}')
#pprint(geelayers.CITY_DATA.sort('UC_NM_MN').aggregate_array('UC_NM_MN').getInfo())

Image Collection Size: 4241
DEST: projects/wri-datalab/cities/urban_land_use/data/test_tori_Apr2024/builtup_density_JRCs_50compare_L2_20_wP


In [9]:
geelayers.CITY_DATA = geelayers.CITY_DATA.sort('P15', False)
OFFSET = config.OFFSET
LIMIT = config.LIMIT

if OFFSET and LIMIT:
    LIMIT = LIMIT+OFFSET
IDS = geelayers.CITY_DATA.aggregate_array('ID_HDC_G0').getInfo()[OFFSET:LIMIT]

pprint(len(IDS))

12844


In [ ]:
# Map the function over the feature collection
geelayers.CITY_DATA = geelayers.CITY_DATA.map(helpers.replace_geometry)

In [ ]:
# Get list of cities
CITIES_LIST = cities.id_hdc_g0_250[1:2]
CITIES_LIST

In [ ]:
# Calculate Urban Extent for a list of cities
super_rasters.get_urban_extents(IDS, CITIES_LIST)

In [ ]:
# Filter to limit cities to be run to those not already in the output ImageCollection.
COMPLETED_IDS = ee.ImageCollection(config.IC_ID).aggregate_array('ID_HDC_G0')
COMPLETED_FILTER = ee.Filter.And(ee.Filter.inList('ID_HDC_G0', COMPLETED_IDS), ee.Filter.equals('builtup_year', config.mapYear))
COMPLETED_CITIES_LIST = ee.ImageCollection(config.IC_ID).filter(COMPLETED_FILTER).aggregate_array('ID_HDC_G0')

if config.USE_COMPLETED_FILTER:
    geelayers.CITY_DATA = geelayers.CITY_DATA.filter(ee.Filter.inList('ID_HDC_G0', COMPLETED_CITIES_LIST).Not())
else:
    geelayers.CITY_DATA = geelayers.CITY_DATA

pprint(geelayers.CITY_DATA.aggregate_array('UC_NM_MN').getInfo())
pprint(geelayers.CITY_DATA.aggregate_array('GRGN_L2').getInfo())
pprint(geelayers.CITY_DATA.aggregate_array('ID_HDC_G0').getInfo())